### Instalando o Experta

In [1]:
!pip install experta

  Created wheel for frozendict: filename=frozendict-1.2-cp37-none-any.whl size=3150 sha256=6b04e0883feabc9c142b9b6de74c284922c0418078bab3cfbeaafd247189e51f
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built frozendict


### Importando dependencias

In [2]:
from experta import *

### Funções uteis

In [4]:
# Funçao auxiliar para receber inputs do teclado
def ask(question, true_value=True, false_value=False):
    while True:
        answer = input(question + "(S/N) ").upper()
        if (answer == 'S'): return true_value
        if (answer == 'N'): return false_value
        print("Responda com 'S' para Sim ou 'N' para Não.")

def ask_feature(question, true_value=True, false_value=False):
    return ask("O animal " + question, true_value, false_value)

# EXEMPLO
print(ask_feature("amamenta?", "mamífero", "não é mamífero"))

O animal amamenta?(S/N) G
Responda com 'S' para Sim ou 'N' para Não.
O animal amamenta?(S/N) S
mamífero


In [5]:
# Definindo tipos de fatos do dominio
class Classe(Fact):
    pass

class Feature(Fact):
    pass

class Animal(Fact):
    pass

# Aliases de prioridade
HIGHEST = 3
HIGH = 2 
MEDIUM = 1
DEFAULT = 0
LOWEST = -1

## ENGINE QUE IDENTIFICA A CLASSE DE UM ANIMAL

In [6]:
class IdentificaClasseAnimal(KnowledgeEngine):
    # FEATURES PARA DEFINIR CLASSE

    @Rule(NOT(Classe(W())))
    def ask_amamenta(self):
        self.declare(Feature(amamenta=ask_feature("é amamentado?")))

    @Rule(NOT(Classe(W())))
    def ask_branqueas(self):
        self.declare(Feature(branqueas=ask_feature("possui brânqueas (Guelras)?")))
    
    @Rule(NOT(Classe(W())))
    def ask_ovo_casca(self):
        self.declare(Feature(ovo_casca=ask_feature("bota ovos com casca rígida?")))
    
    @Rule(NOT(Classe(W())), salience=LOWEST)
    def ask_duas_fases(self):
        self.declare(Feature(duas_fases=ask_feature("passa o começo da vida na água, e depois vive na terra?")))

    @Rule(NOT(Classe(W())), salience=HIGH)
    def ask_possui_penas(self):
        self.declare(Feature(possui_penas=ask_feature("possui penas?")))

    # REGRAS QUE DEFINEM CLASSES

    @Rule(Feature(amamenta=True), salience=HIGHEST)
    def is_mamal(self):
        self.declare(Classe('Mamífero'))

    @Rule(Feature(branqueas=True), salience=HIGHEST)
    def is_fish(self):
        self.declare(Classe('Peixe'))
    
    @Rule(Feature(duas_fases=True), salience=HIGHEST)
    def is_amphibian(self):
        self.declare(Classe('Anfibio'))
    
    @Rule(Feature(possui_penas=True),salience=HIGHEST)
    def is_aves(self):
        self.declare(Classe("Ave"))
    
    @Rule(AND(Feature(ovo_casca=True),Feature(possui_penas=False)),salience=HIGHEST)
    def is_repitile(self):
        self.declare(Classe("Réptil"))

    # FAZENDO MATCH COM A CLASSE DESCOBERTA
    @Rule(Classe(MATCH.name))
    def guess_classe(self, name):
        print("O seu animal é um(a) %s" % (name))

In [7]:
# EXEMPLO ENGINE CLASSE ANIMAL:
engine_classe = IdentificaClasseAnimal()
engine_classe.reset()
engine_classe.run()
engine_classe.facts

O animal possui penas?(S/N) N
O animal é amamentado?(S/N) N
O animal possui brânqueas (Guelras)?(S/N) S
O seu animal é um(a) Peixe


FactList([(0, InitialFact()),
          (1, Feature(possui_penas=False)),
          (2, Feature(amamenta=False)),
          (3, Feature(branqueas=True)),
          (4, Classe('Peixe'))])

# ENGINE QUE IDENTIFICA UM ANIMAL 
### (HERDANDO A ENGINE QUE DEFINE A CLASSE)

In [8]:
class IdentificaAnimal(IdentificaClasseAnimal):
    # COLETANDO FEATURES PARA DEFINIR O ANIMAL:

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")), 
               NOT(Animal(W())) ))
    def ask_carnivoro(self):
        self.declare(Feature(carnivoro=ask_feature("é carnívoro?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")),
               NOT(Animal(W()))), salience=MEDIUM)
    def ask_aquatico_ou_semi(self):
        self.declare(Feature(aquatico_ou_semi=ask_feature("é aquatico ou precisa de viver perto da água?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Mamífero")),
               NOT(Animal(W()))), salience=MEDIUM)
    def ask_voa(self):
        self.declare(Feature(voa=ask_feature("é capaz de voar?")))

    @Rule(AND( Classe("Mamífero"), 
               NOT(Animal(W())) ))
    def ask_selvagem(self):
        self.declare(Feature(selvagem=ask_feature("é selvagem?")))

    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_cartilaginoso(self):
        self.declare(Feature(cartilaginoso=ask_feature("possui esqueleto cartilaginoso?")))
    
    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_fusiforme(self):
        self.declare(Feature(fusiforme=ask_feature("tem forma alongada (fusiforme)?")))

    @Rule(AND( Classe("Anfibio"), 
               NOT(Animal(W())) ))
    def ask_rabo(self):
        self.declare(Feature(rabo=ask_feature("possui rabo?")))

    # REGRAS PARA DIFERENCIAR RÉPTEIS

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_jacare(self):
        self.declare(Animal("Jacaré"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_cobra(self):
        self.declare(Animal("Cobra"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_tartaruga(self):
        self.declare(Animal("Tartaruga"))

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_camaleao(self):
        self.declare(Animal("Camaleão"))

    # REGRAS PARA DIFERENCIAR PEIXES

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=True)), salience=HIGHEST)
    def is_tubarao(self):
        self.declare(Animal("Tubarão"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=False)), salience=HIGHEST)
    def is_arraia(self):
        self.declare(Animal("Arraia"))

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=True)), salience=HIGHEST)
    def is_atum(self):
        self.declare(Animal("Atum"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=False)), salience=HIGHEST)
    def is_baiacu(self):
        self.declare(Animal("Baiacu"))

    # REGRAS PARA DIFERENCIAR AVES

    @Rule(AND(Classe("Ave"), Feature(voa=False)), salience=HIGHEST)
    def is_pinguim(self):
        self.declare(Animal("Pinguim"))
    
    @Rule(AND(Classe("Ave"), Feature(carnivoro=False)), salience=HIGHEST)
    def is_beija_flor(self):
        self.declare(Animal("Beija-flor"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=True), Feature(voa=True)), salience=HIGHEST)
    def is_gaivota(self):
        self.declare(Animal("Gaivota"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_gaviao(self):
        self.declare(Animal("Gavião"))

    # REGRAS PARA DIFERENCIAR MAMIFEROS

    @Rule(AND(Classe("Mamífero"), Feature(voa=True)), salience=HIGHEST)
    def is_morcego(self):
        self.declare(Animal("Morcego"))
    
    @Rule(AND(Classe("Mamífero"), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_baleia(self):
        self.declare(Animal("Baleia"))

    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=False)), salience=HIGHEST)
    def is_humano(self):
        self.declare(Animal("Humano"))
    
    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_cachorro(self):
        self.declare(Animal("Cachorro"))

    @Rule(AND(Classe("Mamífero"), 
            Feature(selvagem=True), 
            Feature(carnivoro=True)), 
        salience=HIGHEST)
    def is_urso(self):
        self.declare(Animal("Urso"))

    # REGRAS PARA DIFERENCIAR ANFIBIOS

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=True)), salience=HIGHEST)
    def is_salamandra(self):
        self.declare(Animal('Salamandra'))

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=False)), salience=HIGHEST)
    def is_sapo(self):
        self.declare(Animal('Sapo'))


    # FAZENDO MATCH COM O ANIMAL DESCOBERTO
    @Rule(Animal(MATCH.name))
    def guess_animal(self, name):
        print("O seu animal é um(a) %s" % (name))

    # Sobrescrevendo guess_classe() para não printar a classe
    def guess_classe(self):
        pass

# MAIN LOOP: IDENTIFICAR ANIMAIS

In [9]:
engine = IdentificaAnimal()
run = ask("Olá! Vamos identificar um animal?")
while(run):
    engine.reset()
    engine.run()
    run = ask("Deseja identificar outro animal?")

Olá! Vamos identificar um animal?(S/N) S
O animal possui penas?(S/N) N
O animal é amamentado?(S/N) N
O animal possui brânqueas (Guelras)?(S/N) N
O animal bota ovos com casca rígida?(S/N) S
O animal é aquatico ou precisa de viver perto da água?(S/N) S
O animal é carnívoro?(S/N) N
O seu animal é um(a) Tartaruga
Deseja identificar outro animal?(S/N) N


### FATOS DA ULTIMA EXECUÇÃO:

In [10]:
engine.facts

FactList([(0, InitialFact()),
          (1, Feature(possui_penas=False)),
          (2, Feature(amamenta=False)),
          (3, Feature(branqueas=False)),
          (4, Feature(ovo_casca=True)),
          (5, Classe('Réptil')),
          (6, Feature(aquatico_ou_semi=True)),
          (7, Feature(carnivoro=False)),
          (8, Animal('Tartaruga'))])

## Herdando a engine IdentificaAnimal somente para demonstrar a inclusão de um leão
#### Das features definidas, urso é o animal que tem caracteristicas idênticas à de um leão, necessitando apenas definir uma nova feature para diferencia-los.

In [11]:
class IdentificaAnimalComLeao(IdentificaAnimal):
    # Nova caracteristica: hiberna.
    @Rule(AND( Classe("Mamífero"), 
               NOT(Animal(W())) ))
    def ask_hiberna(self):
        self.declare(Feature(hiberna=ask_feature("hiberna (geralmente no inverno)?")))

    # Leão tem as características parecidas com a de um urso, mas não hiberna.
    @Rule(AND(Classe("Mamífero"), 
        Feature(selvagem=True), 
        Feature(carnivoro=True), 
        Feature(hiberna=False)), # É A MESMA REGRA QUE O URSO TINHA, COM APENAS ESSA LINHA A MAIS
    salience=HIGHEST)
    def is_leao(self):
        self.declare(Animal("Leão"))

    # SOBRESCREVE is_urso() ADICIONANDO A FEATURE QUE DIFERENCIA ELE DO LEÃO
    # Urso é o único animal que hiberna da base de conhecimento até então.
    @Rule(AND(Classe("Mamífero"), 
            Feature(hiberna=True)), 
        salience=HIGHEST)
    def is_urso(self):
        self.declare(Animal("Urso"))



In [12]:
# EXEMPLO COM LEÃO:
engine = IdentificaAnimalComLeao()
engine.reset()
engine.run()

O animal possui penas?(S/N) N
O animal é amamentado?(S/N) S
O animal é capaz de voar?(S/N) N
O animal é aquatico ou precisa de viver perto da água?(S/N) N
O animal é selvagem?(S/N) S
O animal é carnívoro?(S/N) S
O animal hiberna (geralmente no inverno)?(S/N) S
O seu animal é um(a) Urso
